# 基于MindSpore通过GPT实现情感分类

## 环境配置

    python =3.9
    mindspore = 2.3.1
    mindnlp = 0.4.0
    jieba
    tiktoken

**在线运行代码平台链接：**
- 1. [华为云AI Gallery](https://pangu.huaweicloud.com/gallery/asset-detail.html?id=1325b719-fc78-46c4-8f47-9f3623e9b0f4)
- 2. [大模型平台AI实验室统一入口](https://xihe.mindspore.cn/projects)

## 代码开发

In [4]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn

from mindnlp.dataset import load_dataset

from mindnlp.engine import Trainer

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the default dict

载入与处理数据集

In [5]:
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']

In [6]:
imdb_train.get_dataset_size()

25000

In [7]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=4, shuffle=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'attention_mask': (None, 0)})

    return dataset

载入tokenizer并添加token，用于标记开始（bos）、结束（eos）与填充（pad）。

In [8]:
from mindnlp.transformers import OpenAIGPTTokenizer
# tokenizer
gpt_tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


由于原数据集没有验证集，故将原训练集重新划分为训练集与验证集

In [9]:
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

In [10]:
dataset_train = process_dataset(imdb_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, gpt_tokenizer)
dataset_test = process_dataset(imdb_test, gpt_tokenizer)

In [11]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[4, 512], dtype=Int64, value=
 [[  249,  1275,   239 ...   788,   793,   239],
  [  244,  4867,   498 ... 40480, 40480, 40480],
  [  249,  2496,   763 ... 40480, 40480, 40480],
  [  963,  3666,   239 ... 40480, 40480, 40480]]),
 Tensor(shape=[4, 512], dtype=Int64, value=
 [[1, 1, 1 ... 1, 1, 1],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[4], dtype=Int32, value= [0, 1, 0, 1])]

加载GPT序列分类模型，设置为二分类

In [12]:
from mindnlp.transformers import OpenAIGPTForSequenceClassification

# set GPT config and define parameters for training
model = OpenAIGPTForSequenceClassification.from_pretrained('openai-gpt', num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)



[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of OpenAIGPTForSequenceClassification were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


使用TrainArguments定义训练参数

In [13]:
from mindnlp.engine import TrainingArguments

training_args = TrainingArguments(
    output_dir="gpt_imdb_finetune",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    num_train_epochs=1.0
)


设置训练与评估指标

In [14]:
from mindnlp import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [15]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val, 
    compute_metrics=compute_metrics
)


In [16]:
# start training
trainer.train()

  0%|          | 0/4375 [00:00<?, ?it/s]

{'loss': 0.3937, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/1875 [00:00<?, ?it/s]

{'eval_loss': 0.30218613147735596, 'eval_accuracy': 0.9269333333333334, 'eval_runtime': 131.1926, 'eval_samples_per_second': 14.292, 'eval_steps_per_second': 1.791, 'epoch': 1.0}
{'train_runtime': 1615.1547, 'train_samples_per_second': 10.835, 'train_steps_per_second': 2.709, 'train_loss': 0.3937192522321429, 'epoch': 1.0}


TrainOutput(global_step=4375, training_loss=0.3937192522321429, metrics={'train_runtime': 1615.1547, 'train_samples_per_second': 10.835, 'train_steps_per_second': 2.709, 'train_loss': 0.3937192522321429, 'epoch': 1.0})

In [17]:
trainer.evaluate(dataset_test)

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.30381375551223755,
 'eval_accuracy': 0.92812,
 'eval_runtime': 427.6338,
 'eval_samples_per_second': 14.615,
 'eval_steps_per_second': 1.829,
 'epoch': 1.0}